In [18]:
import pandas as pd
from io import StringIO
import numpy as np
from collections import defaultdict

In [14]:
nexmoLogSample = '''2015-12-20T16:41:39-05:00|EVENT|log=EVENT,MySessionID=895426ce56fa28d2d5b68830087fa58d,MyException=HANGUP,MyEvent=connection.disconnect.hangup,
2015-12-20T16:41:49-05:00|OK|log=OK,req_id=85360453,call-id=edc7d383619194172273a5167494bce9-1,status=ok,to=1234567819,call-request=2015/12/20 21:40:50,call-direction=out,call-price=0.121333,call-rate=0.14000000,call-duration=52,network-code=72406,call-start=2015/12/20 21:40:58,call-end=2015/12/20 21:41:49,
2015-12-20T16:41:51-05:00|OK|log=OK,req_id=84409064,call-id=48c395150ebdf89be85b8f0c775ea3ad-1,status=ok,to=12345678910,call-request=2015/12/20 21:40:08,call-direction=out,call-price=0.121333,call-rate=0.14000000,call-duration=52,network-code=72402,call-start=2015/12/20 21:41:00,call-end=2015/12/20 21:41:51,
2015-12-20T16:41:56-05:00|OK|log=OK,req_id=88425994,call-id=62f11ae821fb18e9ed2fe083f7711d5a-1,status=ok,to=12345678910,call-request=2015/12/20 21:40:35,call-direction=out,call-price=0.135200,call-rate=0.15600000,call-duration=52,network-code=74800,call-start=2015/12/20 21:41:05,call-end=2015/12/20 21:41:56,
2015-12-20T16:42:03-05:00|ERROR|log=ERROR,req_id=83525190,call-id=3c65ccd164422fde2f76c5d40ba1eb72-1,status=failed,sip-status=0,to=12345678910,call-request=2015/12/20 21:41:01,
2015-12-20T16:42:04-05:00|EVENT|log=EVENT,MySessionID=b0068ad9ef85344555c194184993c1ec,MyException=HANGUP,MyEvent=connection.disconnect.hangup,
2015-12-20T16:42:07-05:00|OK|log=OK,req_id=87340519,call-id=537c18322c7a8a9ad123702d83013196-1,status=ok,to=12345678910,call-request=2015/12/20 21:40:40,call-direction=out,call-price=0.135200,call-rate=0.15600000,call-duration=52,network-code=74800,call-start=2015/12/20 21:41:16,call-end=2015/12/20 21:42:07,
2015-12-20T17:06:01-0500|INFO|msg=Translated message is not avaialble.Trying English,req_id=5555555,phone=12345678910,msg_type=TYPE1,lang=zh-cn
2015-12-20T17:06:02-0500|INFO|msg=successful call request,req_id=5555555,phone=12345678910,msg_type=TYPE1,lang=zh-cn,response=HTTP::Response=HASH(0x315d6d8)
2015-12-20T17:06:03-05:00|EVENT|log=EVENT,MySessionID=46b8c692e5ac1b92a956f7846539dab0,CurrentLocation=startMessage,
2015-12-20T17:06:14-05:00|EVENT|log=EVENT,MySessionID=46b8c692e5ac1b92a956f7846539dab0,CurrentLocation=endMessage,
2015-12-20T17:06:30-05:00|OK|log=OK,req_id=5555555,call-id=d363a48c9d7ef7fd394c75aafcd8ec84-1,status=ok,to=12345678910,call-request=2015/12/20 22:06:02,call-direction=out,call-price=0.007933,call-rate=0.02800000,call-duration=17,network-code=CH-FIXED,call-start=2015/12/20 22:06:13,call-end=2015/12/20 22:06:30,
2015-12-20T17:08:21-0500|INFO|msg=successful call request,req_id=5555555,phone=12345678910,msg_type=TYPE1,lang=zh-cn,response=HTTP::Response=HASH(0x12285d8)
2015-12-20T17:08:23-05:00|ERROR|log=ERROR,req_id=5555555,call-id=122761a9cc4c76a8a93a31dbce2a7cb9-1,status=vxml_error,sip-status=500,to=12345678910,call-request=2015/12/20 22:08:20,
2015-12-20T17:09:33-0500|INFO|msg=successful call request,req_id=5555555,phone=12345678910,msg_type=TYPE1,lang=zh-ch,response=HTTP::Response=HASH(0x2b5c5d8)
2015-12-20T17:09:35-05:00|ERROR|log=ERROR,req_id=5555555,call-id=e36bf6a3cbd3f6105fe926699ab02518-1,status=vxml_error,sip-status=500,to=12345678910,call-request=2015/12/20 22:09:33,
2015-12-20T17:21:52-0500|INFO|msg=successful call request,req_id=5555555,phone=12345678910,msg_type=TYPE1,lang=zh,response=HTTP::Response=HASH(0x2e165d8)
2015-12-20T17:21:54-05:00|ERROR|log=ERROR,req_id=5555555,call-id=83249d585dd9ff02a9ac4481506d173b-1,status=vxml_error,sip-status=500,to=12345678910,call-request=2015/12/20 22:21:52,
2015-12-20T17:23:33-0500|INFO|msg=successful call request,req_id=2222222,phone=12345678910,msg_type=TYPE1,lang=en,response=HTTP::Response=HASH(0x30eb5d8)
2015-12-20T17:23:33-05:00|EVENT|log=EVENT,MySessionID=55c3a8304ae02afe153b04e24ad746aa,CurrentLocation=startMessage,
2015-12-20T17:23:45-05:00|EVENT|log=EVENT,MySessionID=55c3a8304ae02afe153b04e24ad746aa,CurrentLocation=endMessage,
2015-12-20T17:23:50-05:00|OK|log=OK,req_id=2222222,call-id=08cfa31d667e2dbac0946c220cb44f18-1,status=ok,to=12345678910,call-request=2015/12/20 22:23:32,call-direction=out,call-price=0.002800,call-rate=0.02800000,call-duration=6,network-code=CH-FIXED,call-start=2015/12/20 22:23:45,call-end=2015/12/20 22:23:50,
2015-12-20T17:35:42-0500|INFO|msg=successful call request,req_id=5555555,phone=12345678910,msg_type=TYPE1,lang=zh,response=HTTP::Response=HASH(0x1ec65d8)
2015-12-20T17:35:43-05:00|ERROR|log=ERROR,req_id=5555555,call-id=343e06d3066f18379c74fc666c5f24a3-1,status=vxml_error,sip-status=500,to=12345678910,call-request=2015/12/20 22:35:42,
2015-12-20T17:51:38-0500|INFO|msg=successful call request,req_id=5555555,phone=12345678910,msg_type=TYPE1,lang=zh,response=HTTP::Response=HASH(0x2c735d8)
2015-12-20T17:51:41-05:00|ERROR|log=ERROR,req_id=5555555,call-id=01c00753bc7daea0caec8f290f1815a1-1,status=vxml_error,sip-status=500,to=12345678910,call-request=2015/12/20 22:51:38,
2015-12-20T18:43:13-0500|INFO|msg=successful call request,req_id=5555555,phone=12345678910,msg_type=TYPE1,lang=zh,response=HTTP::Response=HASH(0x13f15d8)
2015-12-20T18:43:13-05:00|ERROR|log=ERROR,req_id=5555555,call-id=4c25dc4ed18339b03383077b8c1caae1-1,status=vxml_error,sip-status=500,to=12345678910,call-request=2015/12/20 23:43:12,
2015-12-20T18:47:26-0500|ERROR|msg=Failed call request,req_id=5555555,phone=123456789,msg_type=TYPE1,lang=zh,response=HTTP::Response=HASH(0x155c560)
2015-12-20T18:50:54-0500|ERROR|msg=Failed call request,req_id=5555555,phone=1234567891,msg_type=TYPE1,lang=zh,response=HTTP::Response=HASH(0x297f560)
2015-12-20T18:59:17-0500|ERROR|msg=Failed call request,req_id=4444444,phone=12345678910,msg_type=TYPE1,lang=en,response=HTTP::Response=HASH(0x2423560)'''

In [15]:
df = pd.read_csv(StringIO(nexmoLogSample), sep='|', header=None)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 30
Data columns (total 3 columns):
0    31 non-null object
1    31 non-null object
2    31 non-null object
dtypes: object(3)
memory usage: 620.0+ bytes


In [17]:
df.head()

,0,1,2
0,2015-12-20T16:41:39-05:00,EVENT,"log=EVENT,MySessionID=895426ce56fa28d2d5b68830..."
1,2015-12-20T16:41:49-05:00,OK,"log=OK,req_id=85360453,call-id=edc7d3836191941..."
2,2015-12-20T16:41:51-05:00,OK,"log=OK,req_id=84409064,call-id=48c395150ebdf89..."
3,2015-12-20T16:41:56-05:00,OK,"log=OK,req_id=88425994,call-id=62f11ae821fb18e..."
4,2015-12-20T16:42:03-05:00,ERROR,"log=ERROR,req_id=83525190,call-id=3c65ccd16442..."


In [20]:
#We are interested in parsing the last column
df[2].tail()

26    msg=successful call request,req_id=5555555,pho...
27    log=ERROR,req_id=5555555,call-id=4c25dc4ed1833...
28    msg=Failed call request,req_id=5555555,phone=1...
29    msg=Failed call request,req_id=5555555,phone=1...
30    msg=Failed call request,req_id=4444444,phone=4...
Name: 2, dtype: object

In [21]:
#split the column by comma and create columns from it
dfNew = df[2].str.split(',').apply(pd.Series)

In [22]:
dfNew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 30
Data columns (total 14 columns):
0     31 non-null object
1     31 non-null object
2     31 non-null object
3     31 non-null object
4     27 non-null object
5     24 non-null object
6     13 non-null object
7     13 non-null object
8     6 non-null object
9     6 non-null object
10    6 non-null object
11    6 non-null object
12    6 non-null object
13    6 non-null object
dtypes: object(14)
memory usage: 1.9+ KB


In [23]:
dfNew.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,log=EVENT,MySessionID=895426ce56fa28d2d5b68830087fa58d,MyException=HANGUP,MyEvent=connection.disconnect.hangup,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,log=OK,req_id=85360453,call-id=edc7d383619194172273a5167494bce9-1,status=ok,to=5515998563913,call-request=2015/12/20 21:40:50,call-direction=out,call-price=0.121333,call-rate=0.14000000,call-duration=52,network-code=72406,call-start=2015/12/20 21:40:58,call-end=2015/12/20 21:41:49,
2,log=OK,req_id=84409064,call-id=48c395150ebdf89be85b8f0c775ea3ad-1,status=ok,to=554799859493,call-request=2015/12/20 21:40:08,call-direction=out,call-price=0.121333,call-rate=0.14000000,call-duration=52,network-code=72402,call-start=2015/12/20 21:41:00,call-end=2015/12/20 21:41:51,
3,log=OK,req_id=88425994,call-id=62f11ae821fb18e9ed2fe083f7711d5a-1,status=ok,to=59899253560,call-request=2015/12/20 21:40:35,call-direction=out,call-price=0.135200,call-rate=0.15600000,call-duration=52,network-code=74800,call-start=2015/12/20 21:41:05,call-end=2015/12/20 21:41:56,
4,log=ERROR,req_id=83525190,call-id=3c65ccd164422fde2f76c5d40ba1eb72-1,status=failed,sip-status=0,to=557187785719,call-request=2015/12/20 21:41:01,,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
#identify all the column names (left of the equal sign)
logCols = set()
for c in dfNew.columns:
    logCols.update(list(dfNew[c].str.split('=').str.get(0).unique()))

In [26]:
#all available columns within the nexmolog
logCols

{'',
 nan,
 'MyException',
 'msg_type',
 'MySessionID',
 'call-start',
 'network-code',
 'call-rate',
 'phone',
 'CurrentLocation',
 'call-direction',
 'call-duration',
 'call-id',
 'req_id',
 'call-request',
 'status',
 'MyEvent',
 'response',
 'call-end',
 'msg',
 'to',
 'lang',
 'call-price',
 'log',
 'sip-status'}

In [27]:
#remove nonesense columns:
logCols.remove(np.nan)
logCols.remove('')

In [30]:
#Initialize dict of lists for storing splited log
logColsDict = defaultdict(list)
for c in dfNew.columns:
    for logCol in logCols:
        if len(dfNew[dfNew[c].notnull()][dfNew[c].str.startswith(logCol+'=').fillna(False)][c]) > 0:
            logColsDict[logCol.replace('-', '_')].append(dfNew[dfNew[c].notnull()][dfNew[c].str.startswith(logCol+'=').fillna(False)][c].str.split('=', 1).str.get(1))

C:\Users\mhristov\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


In [32]:
for k in logColsDict:
    logColsDict[k] = pd.concat(logColsDict[k])
    logColsDict[k].name = k
    df = df.join(logColsDict[k], how='left')

In [41]:
df.rename(columns={'to': 'to_'}, inplace=True)
df.rename(columns={0: 'dt', 1: 'log_type', 2: 'log_body'}, inplace=True)
df['dt'] = pd.to_datetime(df['dt'])

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 30
Data columns (total 26 columns):
dt                 31 non-null datetime64[ns]
log_type           31 non-null object
log_body           31 non-null object
call_end           6 non-null object
MyEvent            2 non-null object
MyException        2 non-null object
status             13 non-null object
msg_type           12 non-null object
MySessionID        6 non-null object
response           11 non-null object
call_direction     6 non-null object
call_request       13 non-null object
log                19 non-null object
CurrentLocation    4 non-null object
call_price         6 non-null object
msg                12 non-null object
phone              12 non-null object
sip_status         7 non-null object
call_rate          6 non-null object
call_duration      6 non-null object
lang               12 non-null object
call_start         6 non-null object
call_id            13 non-null object
to_                13 non-

In [43]:
df.head()

,dt,log_type,log_body,call_end,MyEvent,MyException,status,msg_type,MySessionID,response,...,phone,sip_status,call_rate,call_duration,lang,call_start,call_id,to_,network_code,req_id
0,2015-12-20 21:41:39,EVENT,"log=EVENT,MySessionID=895426ce56fa28d2d5b68830...",NaN,connection.disconnect.hangup,HANGUP,NaN,NaN,895426ce56fa28d2d5b68830087fa58d,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-12-20 21:41:49,OK,"log=OK,req_id=85360453,call-id=edc7d3836191941...",2015/12/20 21:41:49,NaN,NaN,ok,NaN,NaN,NaN,...,NaN,NaN,0.14000000,52,NaN,2015/12/20 21:40:58,edc7d383619194172273a5167494bce9-1,5515998563913,72406,85360453
2,2015-12-20 21:41:51,OK,"log=OK,req_id=84409064,call-id=48c395150ebdf89...",2015/12/20 21:41:51,NaN,NaN,ok,NaN,NaN,NaN,...,NaN,NaN,0.14000000,52,NaN,2015/12/20 21:41:00,48c395150ebdf89be85b8f0c775ea3ad-1,554799859493,72402,84409064
3,2015-12-20 21:41:56,OK,"log=OK,req_id=88425994,call-id=62f11ae821fb18e...",2015/12/20 21:41:56,NaN,NaN,ok,NaN,NaN,NaN,...,NaN,NaN,0.15600000,52,NaN,2015/12/20 21:41:05,62f11ae821fb18e9ed2fe083f7711d5a-1,59899253560,74800,88425994
4,2015-12-20 21:42:03,ERROR,"log=ERROR,req_id=83525190,call-id=3c65ccd16442...",NaN,NaN,NaN,failed,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,3c65ccd164422fde2f76c5d40ba1eb72-1,557187785719,NaN,83525190
